In [2]:
import shutil
shutil.rmtree('/content/sample_data', ignore_errors=True)

# DOST RoBERTa for NSFW word detection

In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
import tempfile
import subprocess
import sys

IMPORT LIBRARIES AND SETUP

In [4]:
def install_requirements():
    required_packages = [
        'transformers[torch]',
        'datasets',
        'torch',
        'pandas',
        'scikit-learn',
        'onnx',
        'onnxruntime',
        'optimum[onnxruntime]',  # ← This enables ORTModelForSequenceClassification
        'tensorflow',
    ]

    for package in required_packages:
        try:
            __import__(package.split('[')[0])
        except ImportError:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [5]:
print("Checking and installing dependencies...")
install_requirements()

Checking and installing dependencies...
Installing scikit-learn...
Installing onnx...
Installing onnxruntime...
Installing optimum[onnxruntime]...


In [6]:
try:
    from optimum.onnxruntime import ORTModelForSequenceClassification
    from optimum.onnxruntime.configuration import OptimizationConfig
    ONNX_AVAILABLE = True
except ImportError:
    print("Optimum ONNX Runtime not available, ONNX export will be limited")
    ONNX_AVAILABLE = False

Optimum ONNX Runtime not available, ONNX export will be limited


In [7]:
print("All dependencies loaded successfully!")

All dependencies loaded successfully!


# LOAD AND PREPARE DATASET

In [8]:
def load_dataset(csv_path='dataset.csv'):
    print(f"Loading dataset from {csv_path}...")

    try:
        df = pd.read_csv(csv_path)
        print(f"Dataset loaded successfully. Shape: {df.shape}")

        # Validate required columns
        required_cols = ['text', 'label']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")

        # Clean and validate data
        df = df.dropna(subset=['text', 'label'])
        df['text'] = df['text'].astype(str)
        df['label'] = df['label'].astype(int)

        # Validate labels are three (0, 1, 2)
        unique_labels = df['label'].unique()
        if not all(label in [0, 1, 2] for label in unique_labels):
            raise ValueError("Labels must be 0 (positive), 1 (neutral), 2 (negative)")

        print(f"Dataset validation complete. Clean shape: {df.shape}")
        print(f"Label distribution:\n{df['label'].value_counts()}")

        return df

    except FileNotFoundError:
        print(f"Error: Dataset file '{csv_path}' not found!")
        print("Creating a sample dataset for demonstration...")
        return create_sample_dataset()

In [9]:
def create_sample_dataset():
    sample_data = {
        'text': [
            # Positive
            'Good job 2', 'Good job 1', 'Good job 3', 'Good job 4', 'Good job 5',
            'Good job 11', 'Good job 12', 'Good job 13', 'Good job 14', 'Good job 15',
            'Good job 21', 'Good job 22', 'Good job 23', 'Good job 24', 'Good job 25',
            'Good job 32', 'Good job 31', 'Good job 33', 'Good job 34', 'Good job 35',
            'Good job 41', 'Good job 42', 'Good job 43', 'Good job 44', 'Good job 45',
            # Neutral
            'hello', 'world', 'computer', 'programming', 'science',
            'family', 'friendship', 'learning', 'knowledge', 'book',
            'art', 'nature', 'technology', 'innovation', 'creativity',
            'happiness', 'success', 'achievement', 'progress', 'yes',
            # NSFW words (examples - replace with actual dataset)
            'inappropriate1', 'inappropriate2', 'inappropriate3', 'inappropriate4',
            'inappropriate5', 'inappropriate6', 'inappropriate7', 'inappropriate8',
            'inappropriate9', 'inappropriate10', 'inappropriate11', 'inappropriate12',
            'inappropriate13', 'inappropriate14', 'inappropriate15', 'inappropriate16',
            'inappropriate17', 'inappropriate18', 'inappropriate19', 'inappropriate20',
            'inappropriate21', 'inappropriate22', 'inappropriate23', 'inappropriate24', 'inappropriate25'
        ],
        'label': [0] * 25 + [1] * 20 + [2] * 25  # 28 positive, 20 neutral, 25 nsfw
    }

    df = pd.DataFrame(sample_data)
    df.to_csv('dataset.csv', index=False)
    print("Sample dataset created and saved as 'dataset.csv'")
    return df

In [10]:
def split_dataset(df, test_size=0.2, random_state=42):
    print(f"Splitting dataset: {test_size*100}% for validation...")

    X_train, X_val, y_train, y_val = train_test_split(
        df['text'].tolist(),
        df['label'].tolist(),
        test_size=test_size,
        random_state=random_state,
        stratify=df['label']
    )

    print(f"Training set: {len(X_train)} samples")
    print(f"Validation set: {len(X_val)} samples")

    return X_train, X_val, y_train, y_val

# TOKENIZATION AND PREPROCESSING

In [11]:
def create_tokenized_datasets(X_train, X_val, y_train, y_val, model_name):
    """Tokenize the datasets using the model tokenizer"""
    print("Loading tokenizer and creating tokenized datasets...")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Create datasets
    train_dataset = Dataset.from_dict({
        'text': X_train,
        'labels': y_train
    })

    val_dataset = Dataset.from_dict({
        'text': X_val,
        'labels': y_val
    })

    def tokenize_function(examples):
        return tokenizer(
            examples['text'],
            truncation=True,
            padding=False,  # Will be handled by data collator
            max_length=128  # Reasonable for word/short phrase detection
        )

    # Tokenize datasets
    train_tokenized = train_dataset.map(tokenize_function, batched=True)
    val_tokenized = val_dataset.map(tokenize_function, batched=True)

    print("Tokenization complete!")
    return train_tokenized, val_tokenized, tokenizer

# MODEL TRAINING

In [12]:
def train_model(train_dataset, val_dataset, tokenizer, model_name, output_dir):
    """Train the RoBERTa-tl model"""
    print("Initializing model for training...")

    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=3,
        id2label={0: "Positive", 1: "Neutral", 2: "NSFW"},
        label2id={"Positive": 0, "Neutral": 1, "NSFW": 2}
    )

    # Data collator
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Create base arguments without the problematic parameters
    training_args_dict = {
        'output_dir': output_dir,
        'num_train_epochs': 5,
        'per_device_train_batch_size': 16,
        'per_device_eval_batch_size': 16,
        'learning_rate': 2e-5,
        'weight_decay': 0.01,
        'logging_dir': f'{output_dir}/logs',
        'logging_steps': 10,
        'save_total_limit': 2,
        'load_best_model_at_end': True,
        'metric_for_best_model': "eval_loss",
        'greater_is_better': False,
        'report_to': [],
        'seed': 42,
        'dataloader_num_workers': 0,
        'remove_unused_columns': True
    }

    # Add version-specific parameters
    if hasattr(TrainingArguments, 'eval_strategy'):
        # New version
        training_args_dict['eval_strategy'] = "epoch"
        training_args_dict['save_strategy'] = "epoch"
    else:
        # Old version
        training_args_dict['evaluation_strategy'] = "epoch"
        training_args_dict['save_strategy'] = "epoch"

    training_args = TrainingArguments(**training_args_dict)

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        return {
            'accuracy': accuracy_score(labels, predictions),
        }

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print("Starting training...")
    trainer.train()

    # Save the best model
    print(f"Saving model to {output_dir}...")
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)

    return trainer, model

# MODEL EVALUATION

In [13]:
def evaluate_model(trainer, X_val, y_val, output_dir):
    print("Evaluating model performance...")

    # Get predictions
    eval_results = trainer.evaluate()

    # Get detailed predictions for classification report
    predictions = trainer.predict(trainer.eval_dataset)
    y_pred = np.argmax(predictions.predictions, axis=1)

    # Generate classification report
    report = classification_report(
        y_val,
        y_pred,
        target_names=['Positive', 'Neutral', 'NSFW'],
        digits=4
    )

    # Calculate additional metrics
    accuracy = accuracy_score(y_val, y_pred)

    # Prepare metrics text
    metrics_text = f"""NSFW Detection Model Evaluation Results
{'='*50}

Accuracy: {accuracy:.4f}

Classification Report:
{report}

Training Results:
{'-'*30}
"""

    for key, value in eval_results.items():
        metrics_text += f"{key}: {value:.4f}\n"

    # Save metrics
    os.makedirs('metrics', exist_ok=True)
    metrics_path = 'metrics/metrics.txt'

    with open(metrics_path, 'w') as f:
        f.write(metrics_text)

    print(f"Metrics saved to {metrics_path}")
    print(f"Validation Accuracy: {accuracy:.4f}")

    return accuracy, report

# ONNX EXPORT

In [14]:
def export_to_onnx(model_dir, onnx_path):
    print("Exporting model to ONNX format...")

    try:
        # Load the trained model
        model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        tokenizer = AutoTokenizer.from_pretrained(model_dir)

        # Create dummy input
        dummy_input = tokenizer(
            "sample text",
            return_tensors="pt",
            max_length=128,
            padding="max_length",
            truncation=True
        )

        # Export to ONNX with newer opset version
        os.makedirs(os.path.dirname(onnx_path), exist_ok=True)
        torch.onnx.export(
            model,
            tuple(dummy_input.values()),
            onnx_path,
            export_params=True,
            opset_version=14,  # ← Changed from 11 to 14
            do_constant_folding=True,
            input_names=['input_ids', 'attention_mask'],
            output_names=['logits'],
            dynamic_axes={
                'input_ids': {0: 'batch_size'},
                'attention_mask': {0: 'batch_size'},
                'logits': {0: 'batch_size'}
            }
        )

        print(f"ONNX model exported to: {onnx_path}")
        return True

    except Exception as e:
        print(f"ONNX export failed: {e}")
        return False

# TENSORFLOW LITE EXPORT

In [15]:
def export_to_tflite_from_pt(model_dir, tflite_path):
    try:
        import tensorflow as tf
        from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
        import os

        print("Converting PyTorch model directly to TensorFlow...")

        # Load and convert to TensorFlow
        tokenizer = AutoTokenizer.from_pretrained(model_dir)
        tf_model = TFAutoModelForSequenceClassification.from_pretrained(
            model_dir,
            from_pt=True  # convert from PyTorch
        )

        max_len = 256

        @tf.function(input_signature=[
            tf.TensorSpec([None, max_len], tf.int32, name="input_ids"),
            tf.TensorSpec([None, max_len], tf.int32, name="attention_mask")
        ])
        def serving_fn(input_ids, attention_mask):
            outputs = tf_model(input_ids, attention_mask=attention_mask, training=False)
            return {"logits": outputs.logits}

        # Save as TensorFlow SavedModel
        tf_saved_model_dir = os.path.join(model_dir, "tf_saved_model")
        tf.saved_model.save(tf_model, tf_saved_model_dir, signatures={"serving_default": serving_fn})
        print(f"Saved intermediate TensorFlow model to {tf_saved_model_dir}")

        # Convert to TFLite
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_saved_model_dir)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]  # enable optimization
        tflite_model = converter.convert()

        # Save TFLite model
        os.makedirs(os.path.dirname(tflite_path), exist_ok=True)
        with open(tflite_path, "wb") as f:
            f.write(tflite_model)

        print(f"TFLite model successfully exported to: {tflite_path}")
        return True

    except Exception as e:
        print(f"Direct TensorFlow Lite export failed: {e}")
        return False


# MAIN

In [16]:
def main():
    print("Starting NSFW Detection Model Training Pipeline")
    print("="*60)

    # Configuration
    MODEL_NAME = "dost-asti/RoBERTa-tl-sentiment-analysis"  # RoBERTa-tl model
    OUTPUT_DIR = "models/roberta_tl_nsfw"
    ONNX_PATH = "models/nsfw_model.onnx"
    TFLITE_PATH = "models/nsfw_model.tflite"

    # Create output directories
    os.makedirs("models", exist_ok=True)
    os.makedirs("metrics", exist_ok=True)

    # Step 1: Load dataset
    df = load_dataset()

    # Step 2: Split dataset
    X_train, X_val, y_train, y_val = split_dataset(df)

    # Step 3: Create tokenized datasets
    train_dataset, val_dataset, tokenizer = create_tokenized_datasets(
        X_train, X_val, y_train, y_val, MODEL_NAME
    )

    # Step 4: Train model
    trainer, model = train_model(
        train_dataset, val_dataset, tokenizer, MODEL_NAME, OUTPUT_DIR
    )

    # Step 5: Evaluate model
    accuracy, report = evaluate_model(trainer, X_val, y_val, OUTPUT_DIR)

    # Step 6: Export to ONNX
    onnx_success = export_to_onnx(OUTPUT_DIR, ONNX_PATH)

    # Step 7: Export directly to TFLite from PyTorch
    tflite_success = export_to_tflite_from_pt(OUTPUT_DIR, TFLITE_PATH)

    # Final output
    print("\n" + "="*60)
    print("Export complete!")

    if onnx_success:
        print(f"ONNX model: {ONNX_PATH}")
    else:
        print("ONNX export: FAILED")

    if tflite_success:
        print(f"TFLite model: {TFLITE_PATH}")
    else:
        print("TFLite export: FAILED (ONNX model available)")

    print(f"\nModel checkpoints: {OUTPUT_DIR}")
    print(f"Metrics: metrics/metrics.txt")
    print(f"Final validation accuracy: {accuracy:.4f}")

# INFERENCE

In [17]:
def test_inference(model_dir, test_texts=None):
    if test_texts is None:
        test_texts = ["hello world", "inappropriate example"]

    print("\nTesting trained model...")

    try:
        # Load model and tokenizer
        model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        tokenizer = AutoTokenizer.from_pretrained(model_dir)

        model.eval()

        for text in test_texts:
            # Tokenize
            inputs = tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                max_length=128,
                padding=True
            )

            # Predict
            with torch.no_grad():
              outputs = model(**inputs)
              predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
              predicted_class = torch.argmax(predictions, dim=-1).item()
              confidence = predictions[0][predicted_class].item()

            # Map sentiment to NSFW/SAFE (customize this logic based on your needs)
            # Negative sentiment (0) → NSFW
            # Positive sentiment (2) → SAFE
            # Neutral sentiment (1) → UNCERTAIN/SAFE (you can adjust this)

            if predicted_class == 0:  # NEGATIVE → NSFW
                label = "NSFW"
                nsfw_confidence = predictions[0][0].item()
            elif predicted_class == 2:  # POSITIVE → SAFE
                label = "SAFE"
                safe_confidence = predictions[0][2].item()
            else:  # NEUTRAL → treat as SAFE or create a threshold
                label = "SAFE"  # or "UNCERTAIN"
                safe_confidence = predictions[0][1].item()  # Neutral probability
            print(f"Text: '{text}' -> {label} (confidence: {confidence:.4f})")
            print(f"  Class probabilities: Negative={predictions[0][0]:.4f}, Neutral={predictions[0][1]:.4f}, Positive={predictions[0][2]:.4f}")
    except Exception as e:
        print(f"Inference test failed: {e}")

# PROGRAM EXECUTION

In [18]:
if __name__ == "__main__":
    try:
        # Run the main pipeline
        main()

        # Optional: Test inference
        test_inference("models/roberta_tl_nsfw")

    except KeyboardInterrupt:
        print("\nTraining interrupted by user.")
    except Exception as e:
        print(f"Error during execution: {e}")
        import traceback
        traceback.print_exc()

    print("\nProgram execution completed.")

Starting NSFW Detection Model Training Pipeline
Loading dataset from dataset.csv...
Error: Dataset file 'dataset.csv' not found!
Creating a sample dataset for demonstration...
Sample dataset created and saved as 'dataset.csv'
Splitting dataset: 20.0% for validation...
Training set: 56 samples
Validation set: 14 samples
Loading tokenizer and creating tokenized datasets...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Metrics saved to metrics/metrics.txt
Validation Accuracy: 1.0000
Exporting model to ONNX format...


/tmp/ipython-input-1865692038.py:20: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(
/usr/local/lib/python3.12/dist-packages/transformers/modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause t

ONNX model exported to: models/nsfw_model.onnx
Converting PyTorch model directly to TensorFlow...


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Saved intermediate TensorFlow model to models/roberta_tl_nsfw/tf_saved_model
TFLite model successfully exported to: models/nsfw_model.tflite

Export complete!
ONNX model: models/nsfw_model.onnx
TFLite model: models/nsfw_model.tflite

Model checkpoints: models/roberta_tl_nsfw
Metrics: metrics/metrics.txt
Final validation accuracy: 1.0000

Testing trained model...
Text: 'hello world' -> SAFE (confidence: 0.9999)
  Class probabilities: Negative=0.0000, Neutral=0.9999, Positive=0.0000
Text: 'inappropriate example' -> SAFE (confidence: 0.9998)
  Class probabilities: Negative=0.0001, Neutral=0.0001, Positive=0.9998

Program execution completed.


In [19]:
# Create zip of entire content folder
!zip -r /content/colab_content_dost.zip /content/

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.09.19/ (stored 0%)
  adding: content/.config/logs/2025.09.19/13.39.59.525129.log (deflated 92%)
  adding: content/.config/logs/2025.09.19/13.40.42.120946.log (deflated 56%)
  adding: content/.config/logs/2025.09.19/13.40.32.438351.log (deflated 58%)
  adding: content/.config/logs/2025.09.19/13.40.41.341666.log (deflated 57%)
  adding: content/.config/logs/2025.09.19/13.40.30.940050.log (deflated 87%)
  adding: content/.config/logs/2025.09.19/13.40.21.959342.log (deflated 58%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (store